In [290]:
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd

In [291]:
data_path = "C:/Users/Katta's/PycharmProjects/data/usecase3/train.csv"
housing = pd.read_csv(data_path)

In [292]:
 house_obj= housing.select_dtypes(include=['object']).copy()

In [293]:
house_obj.drop(house_obj.columns[house_obj.isnull().any()],axis=1, inplace=True) 

In [294]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in house_obj.columns.values:
        le.fit(house_obj[col])
        house_obj[col]=le.transform(house_obj[col])        

In [295]:
# All types except object
housing_int = housing.select_dtypes(exclude=['object']).copy()
# housing_cat1 = housing_cat.astype('int64',copy=False)
int_columns = housing_int.columns


In [296]:
from sklearn.preprocessing import StandardScaler
# Scaling training because of variance
ss = StandardScaler()
# scaling training data
housing_cat1_scaled = ss.fit_transform(housing_int)


c:\users\katta's\venv\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\katta's\venv\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [297]:
housing_cat1_scaled_df = pd.DataFrame(housing_cat1_scaled, columns=int_columns)

In [298]:
housing_t = pd.concat([house_obj, housing_cat1_scaled_df],axis= 1)


In [299]:
housing_t['GarageYrBlt'].fillna(housing_t['GarageYrBlt'].mean(), inplace=True)
housing_t['MasVnrArea'].fillna(housing_t['MasVnrArea'].mean(), inplace=True)
housing_t['LotFrontage'].fillna(housing_t['LotFrontage'].mean(), inplace=True)

In [300]:
housing_train = housing_t.dropna()
print(housing_train.columns)

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
       'SaleType', 'SaleCondition', 'Id', 'MSSubClass', 'LotFrontage',
       'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')


In [301]:
train_x = housing_train.drop(['SalePrice', 'Id'],axis=1)
train_y = housing_train['SalePrice']

In [302]:
def baseline_model():
	model = Sequential()
	model.add(Dense(64, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
	return model

In [303]:
# fix random seed for reproducibility
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
seed = 7
np.random.seed(seed)
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)


dict_keys(['explained_variance', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted'])


In [304]:
import sklearn
print(sklearn.metrics.SCORERS.keys())

dict_keys(['explained_variance', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted'])


In [305]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, train_x.values, train_y.values, cv=kfold)

mean_error :  0.09827709863145845
min_error 0.00011189125174693171
max_error 0.9691362279075868
std_error 0.0938246690829176


In [306]:
print(results.mean())
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

mean_error :  0.09827709863145845
min_error 0.00011189125174693171
max_error 0.9691362279075868
std_error 0.0938246690829176


In [308]:
estimator.fit(train_x.values, train_y.values)
prediction = estimator.predict(train_x.values)

mean_error :  0.09827709863145845
min_error 0.00011189125174693171
max_error 0.9691362279075868
std_error 0.0938246690829176


In [309]:
train_error =  np.abs(train_y - prediction)
print('mean_error : ', np.mean(train_error))
print('min_error', np.min(train_error))
print('max_error', np.max(train_error))
print('std_error', np.std(train_error))


mean_error :  0.09827709863145845
min_error 0.00011189125174693171
max_error 0.9691362279075868
std_error 0.0938246690829176
